<a href="https://colab.research.google.com/github/alesouzaeu/Coursera_Capstone/blob/main/Segmenting_and_Clustering_Neighborhoods_in_Toronto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Segmenting and Clustering Neighborhoods in Toronto

## Install the necessaries packages We will need

In [1]:
!pip install requests2
!pip install beautifulsoup4

## Importing the libraries we will need.

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from geopy.geocoders import Nominatim

import numpy as np 
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


#Tirando Warnings do código
import warnings
warnings.filterwarnings('ignore')




Libraries imported.


## Making web scraping with Beautiful Soup 4 in Wikipedia URL 

In [3]:
# I called the URL that contains the table we want.
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
response = requests.get(URL)
soup = BeautifulSoup(response.text, 'html.parser')
# Parseamos o Html desejado em variáveis para facilitar o trabalho
table = soup.find('table', {'class':'wikitable sortable'}).tbody

rows = table.find_all('tr')

columns = [v.text.replace('\n', '') for v in rows[0].find_all('th')]
# Defining the columns names with the web scraped texts.
df =  pd.DataFrame(columns=columns)
# Creating a loop to take a data at the right position in the dataframe.
for i in range(1, len(rows)):
  tds = rows[i].find_all('td')

  if len(tds) ==4:
      values = [tds[0].text, tds[1].text, tds[2].text.replace('\n','')]# eliminamos caracteres indesejados por meio do método .replace
  else:
    values = [td.text.replace('\n','') for td in tds ]# eliminamos caracteres indesejados por meio do método .replace
 
  df = df.append(pd.Series(values, index=columns), ignore_index=True)




In [4]:
# Printing the Header

df_header = pd.DataFrame(columns=columns)
df_header

,Postal Code,Borough,Neighbourhood


In [5]:
# The Dataframe became like that.

df.head(10)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [6]:
# We got out the Dataframe at json format.

df.to_json(r'list_of_postal_code_of_canada_m.json')

In [7]:
# We have imported the table to our notebook.
df = pd.read_json('list_of_postal_code_of_canada_m.json')

In [8]:
df = df.rename(columns={"Neighbourhood":"Neighborhood"})

In [9]:
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [10]:
df.shape

(180, 3)

### Now, We have to throw out the 'Not assigned' data.

In [11]:
df = df.loc[df['Neighborhood']!='Not assigned'] 

In [12]:
df.head(20)

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [13]:
df1 = df

In [14]:
# new data frame with split value columns 
df1["Neighborhood"]= df1["Neighborhood"].str.split(", ", n = 1, expand = True) 

# df display 

In [15]:
pd.set_option("display.max_rows", None, "display.max_columns", None)


In [16]:
df1.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park
5,M6A,North York,Lawrence Manor
6,M7A,Downtown Toronto,Queen's Park


In [17]:
#create variable column inside file called `myAddress and assign it to seleted address columns
df1['Query'] = df1['Postal Code'] + ", " +  df1['Borough']+ ", " + df1['Neighborhood']

#print new column with index -first 5 rows only
df1.head()

,Postal Code,Borough,Neighborhood,Query
2,M3A,North York,Parkwoods,"M3A, North York, Parkwoods"
3,M4A,North York,Victoria Village,"M4A, North York, Victoria Village"
4,M5A,Downtown Toronto,Regent Park,"M5A, Downtown Toronto, Regent Park"
5,M6A,North York,Lawrence Manor,"M6A, North York, Lawrence Manor"
6,M7A,Downtown Toronto,Queen's Park,"M7A, Downtown Toronto, Queen's Park"


### Now We need to insert new columns to our dataset with Latitude and Longitude

In [18]:

#create 2 new columns to store lat/long - initalise to null
df1['Latitude'] = ""
df1['Longitude'] = ""

#print first rows to sample
df1.head()

,Postal Code,Borough,Neighborhood,Query,Latitude,Longitude
2,M3A,North York,Parkwoods,"M3A, North York, Parkwoods",,
3,M4A,North York,Victoria Village,"M4A, North York, Victoria Village",,
4,M5A,Downtown Toronto,Regent Park,"M5A, Downtown Toronto, Regent Park",,
5,M6A,North York,Lawrence Manor,"M6A, North York, Lawrence Manor",,
6,M7A,Downtown Toronto,Queen's Park,"M7A, Downtown Toronto, Queen's Park",,


### Here We are must to find the locations for each Postal Code with the Library GeoPy

In [19]:
'''
**Get Lat/Long Data with GeoPy**
---------------------

the code below calls a geopy API using a concatenated column of address values. We use this column as a query key 
to pull back cooresponding lat/long coordinates.
'''

geolocator = Nominatim(user_agent="geoapiExercises")

for i in df1.index:
    try:
        #tries fetch address from geopy
        location = geolocator.geocode(df1['Query'][i])
        
        #append lat/long to column using dataframe location
        df1.loc[i,'Latitude'] = location.latitude
        df1.loc[i,'Longitude'] = location.longitude
    except:
        #catches exception for the case where no value is returned
        #appends null value to column
        df1.loc[i,'Latitude'] = "null"
        df1.loc[i,'Longitude'] = "null"
#print first rows as sample
df1.head()

,Postal Code,Borough,Neighborhood,Query,Latitude,Longitude
2,M3A,North York,Parkwoods,"M3A, North York, Parkwoods",null,null
3,M4A,North York,Victoria Village,"M4A, North York, Victoria Village",null,null
4,M5A,Downtown Toronto,Regent Park,"M5A, Downtown Toronto, Regent Park",null,null
5,M6A,North York,Lawrence Manor,"M6A, North York, Lawrence Manor",null,null
6,M7A,Downtown Toronto,Queen's Park,"M7A, Downtown Toronto, Queen's Park",null,null


In [20]:
df1 = df1.loc[df1['Latitude']!='null'] 

In [21]:
df1

,Postal Code,Borough,Neighborhood,Query,Latitude,Longitude
9,M1B,Scarborough,Malvern,"M1B, Scarborough, Malvern",43.8092,-79.2217
18,M1C,Scarborough,Rouge Hill,"M1C, Scarborough, Rouge Hill",43.7803,-79.1305
26,M9C,Etobicoke,Eringate,"M9C, Etobicoke, Eringate",43.6623,-79.5765
36,M1G,Scarborough,Woburn,"M1G, Scarborough, Woburn",43.7598,-79.2253
49,M5H,Downtown Toronto,Richmond,"M5H, Downtown Toronto, Richmond",43.6512,-79.3813
68,M6K,West Toronto,Brockton,"M6K, West Toronto, Brockton",43.6509,-79.44
82,M2M,North York,Willowdale,"M2M, North York, Willowdale",43.786,-79.416
91,M2N,North York,Willowdale,"M2N, North York, Willowdale",43.7615,-79.4109
95,M6N,York,Runnymede,"M6N, York, Runnymede",43.6682,-79.4793
104,M6P,West Toronto,High Park,"M6P, West Toronto, High Park",43.6539,-79.4669


In [22]:
df1 = df1.reset_index()

In [23]:
df1.drop(['index'], axis=1)

,Postal Code,Borough,Neighborhood,Query,Latitude,Longitude
0,M1B,Scarborough,Malvern,"M1B, Scarborough, Malvern",43.8092,-79.2217
1,M1C,Scarborough,Rouge Hill,"M1C, Scarborough, Rouge Hill",43.7803,-79.1305
2,M9C,Etobicoke,Eringate,"M9C, Etobicoke, Eringate",43.6623,-79.5765
3,M1G,Scarborough,Woburn,"M1G, Scarborough, Woburn",43.7598,-79.2253
4,M5H,Downtown Toronto,Richmond,"M5H, Downtown Toronto, Richmond",43.6512,-79.3813
5,M6K,West Toronto,Brockton,"M6K, West Toronto, Brockton",43.6509,-79.44
6,M2M,North York,Willowdale,"M2M, North York, Willowdale",43.786,-79.416
7,M2N,North York,Willowdale,"M2N, North York, Willowdale",43.7615,-79.4109
8,M6N,York,Runnymede,"M6N, York, Runnymede",43.6682,-79.4793
9,M6P,West Toronto,High Park,"M6P, West Toronto, High Park",43.6539,-79.4669


### So, This is the shape of our dataframe


In [24]:
df1.shape

(13, 7)

In [25]:
address = 'Toronto, Ontario, CA'

geolocator = Nominatim(user_agent="geoapiExercises")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Define Foursquare Credentials and Version

In [26]:
CLIENT_ID = 'ZYWADMT42YG34PJ0VP35KYUW2O2CP5B4DAHRZAFB50K5CFTA' # your Foursquare ID
CLIENT_SECRET = 'HECPR1B52DIXND5DPGS4ZSRCBULKQ3SXTR0TCUFYROPT2RPX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZYWADMT42YG34PJ0VP35KYUW2O2CP5B4DAHRZAFB50K5CFTA
CLIENT_SECRET:HECPR1B52DIXND5DPGS4ZSRCBULKQ3SXTR0TCUFYROPT2RPX


In [27]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df1['Latitude'], df1['Longitude'], df1['Borough'], df1['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Let's explore the first neighborhood in our dataframe.



In [28]:
df1.loc[0, 'Neighborhood']


'Malvern'

In [29]:
neighborhood_latitude = df1.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df1.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df1.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Malvern are 43.8091955, -79.2217008.


#### Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.

In [30]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius


url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=ZYWADMT42YG34PJ0VP35KYUW2O2CP5B4DAHRZAFB50K5CFTA&client_secret=HECPR1B52DIXND5DPGS4ZSRCBULKQ3SXTR0TCUFYROPT2RPX&v=20180605&ll=43.8091955,-79.2217008&radius=500&limit=100'

In [31]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60334f3794926610d301e150'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-545ecf75498ec51130da75e0-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/pharmacy_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d10f951735',
         'name': 'Pharmacy',
         'pluralName': 'Pharmacies',
         'primary': True,
         'shortName': 'Pharmacy'}],
       'id': '545ecf75498ec51130da75e0',
       'location': {'address': '1400 Neilson Rd',
        'cc': 'CA',
        'city': 'Scarborough',
        'country': 'Canada',
        'distance': 94,
        'formattedAddress': ['1400 Neilson Rd',
         'Scarborough ON M1B 0C2',
         'Canada'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.80961,
    

In [32]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [33]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Shoppers Drug Mart,Pharmacy,43.809610,-79.222729
1,Subway,Sandwich Place,43.806961,-79.221476
2,A&W,Fast Food Restaurant,43.809292,-79.223009
3,Malvern Arena,Skating Rink,43.808594,-79.216634
4,Pizza Pizza,Pizza Place,43.806613,-79.221243


In [34]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

14 venues were returned by Foursquare.


### Explore Neighborhoods in Manhattan

In [35]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called _manhattan_venues_.


In [36]:
toronto_venues = getNearbyVenues(names=df1['Neighborhood'],
                                   latitudes=df1['Latitude'],
                                   longitudes=df1['Longitude']
                                  )

Malvern
Rouge Hill
Eringate
Woburn
Richmond
Brockton
Willowdale
Willowdale
Runnymede
High Park
Kingsview Village
Runnymede
Steeles West


#### Let's check the size of the resulting dataframe

In [37]:
print(toronto_venues.shape)
toronto_venues.head()

(277, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Malvern,43.809196,-79.221701,Shoppers Drug Mart,43.809610,-79.222729,Pharmacy
1,Malvern,43.809196,-79.221701,Subway,43.806961,-79.221476,Sandwich Place
2,Malvern,43.809196,-79.221701,A&W,43.809292,-79.223009,Fast Food Restaurant
3,Malvern,43.809196,-79.221701,Malvern Arena,43.808594,-79.216634,Skating Rink
4,Malvern,43.809196,-79.221701,Pizza Pizza,43.806613,-79.221243,Pizza Place


#### Let's find out how many unique categories can be curated from all the returned venues


In [38]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Brockton,19,19,19,19,19,19
Eringate,2,2,2,2,2,2
High Park,12,12,12,12,12,12
Kingsview Village,1,1,1,1,1,1
Malvern,14,14,14,14,14,14
Richmond,100,100,100,100,100,100
Rouge Hill,2,2,2,2,2,2
Runnymede,38,38,38,38,38,38
Steeles West,1,1,1,1,1,1


In [39]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 102 uniques categories.


## Analyze Each Neighborhood

In [40]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,American Restaurant,Art Gallery,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Beer Bar,Beer Store,Big Box Store,Bookstore,Breakfast Spot,Bubble Tea Shop,Burger Joint,Bus Line,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Electronics Store,Factory,Fast Food Restaurant,Food Court,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gas Station,Gastropub,General Travel,Gluten-free Restaurant,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hot Dog Joint,Hotel,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Karaoke Bar,Korean Restaurant,Latin American Restaurant,Lingerie Store,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Monument / Landmark,Nail Salon,Neighborhood,New American Restaurant,Opera House,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shipping Store,Shopping Mall,Skating Rink,Smoothie Shop,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tanning Salon,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Malvern,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Malvern,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Malvern,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Malvern,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Malvern,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [41]:
toronto_onehot.shape

(277, 102)

In [42]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,American Restaurant,Art Gallery,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Beer Bar,Beer Store,Big Box Store,Bookstore,Breakfast Spot,Bubble Tea Shop,Burger Joint,Bus Line,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Electronics Store,Factory,Fast Food Restaurant,Food Court,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gas Station,Gastropub,General Travel,Gluten-free Restaurant,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hot Dog Joint,Hotel,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Karaoke Bar,Korean Restaurant,Latin American Restaurant,Lingerie Store,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Monument / Landmark,Nail Salon,New American Restaurant,Opera House,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shipping Store,Shopping Mall,Skating Rink,Smoothie Shop,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tanning Salon,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,Brockton,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.052632,0.000000,0.157895,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.052632,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.00,0.052632,0.000000,0.00,0.00,0.000000,0.052632,0.00,0.00,0.00,0.052632,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.052632,0.000000,0.052632,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.105263,0.000000,0.000000,0.052632,0.0,0.00,0.000000,0.000000,0.052632,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.0,0.00,0.000000,0.157895,0.00
1,Eringate,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.500000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.0,0.00,0.000000,0.000000,0.00
2,High Park,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.00,0.166667,0.00,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.083333,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.083333,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.083333,0.000000,0.083333,0.0,0.00,0.000000,0.083333,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.083333,0.00,0.083333,0.00,0.000000,0.00,0.083333,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.0,0.00,0.000000,0.000000,0.00
3,Kingsview Village,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.0000

In [43]:
toronto_grouped.shape

(11, 102)

#### Let's print each neighborhood along with the top 5 most common venues

In [44]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Brockton----
                   venue  freq
0  Vietnamese Restaurant  0.16
1                    Bar  0.16
2                   Park  0.11
3          Grocery Store  0.05
4      French Restaurant  0.05


----Eringate----
                   venue  freq
0                   Park   0.5
1      Electronics Store   0.5
2            Yoga Studio   0.0
3          Poutine Place   0.0
4  Portuguese Restaurant   0.0


----High Park----
                 venue  freq
0    Convenience Store  0.17
1                  Gym  0.08
2       Sandwich Place  0.08
3  Sporting Goods Shop  0.08
4            Pet Store  0.08


----Kingsview Village----
                   venue  freq
0                   Park   1.0
1            Yoga Studio   0.0
2                    Pub   0.0
3          Poutine Place   0.0
4  Portuguese Restaurant   0.0


----Malvern----
                  venue  freq
0  Fast Food Restaurant  0.21
1              Pharmacy  0.14
2           Pizza Place  0.14
3       Bubble Tea Shop  0.07
4        Sandwic

In [45]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [46]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Brockton,Bar,Vietnamese Restaurant,Park,Dive Bar,French Restaurant,Coffee Shop,Gastropub,Pizza Place,Portuguese Restaurant,Bakery
1,Eringate,Electronics Store,Park,Wine Bar,Furniture / Home Store,Diner,Discount Store,Dive Bar,Factory,Fast Food Restaurant,Food Court
2,High Park,Convenience Store,Pool,Gym,Deli / Bodega,Pizza Place,Pet Store,Café,Mexican Restaurant,Sandwich Place,Sporting Goods Shop
3,Kingsview Village,Park,Wine Bar,Furniture / Home Store,Diner,Discount Store,Dive Bar,Electronics Store,Factory,Fast Food Restaurant,Food Court
4,Malvern,Fast Food Restaurant,Pizza Place,Pharmacy,Convenience Store,Grocery Store,Park,Sandwich Place,Supermarket,Bubble Tea Shop,Skating Rink


## Cluster Neighborhoods

In [47]:
# set number of clusters
kclusters = 6

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 2, 1, 0, 1, 1, 4, 1, 3, 1], dtype=int32)

In [48]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df1

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,index,Postal Code,Borough,Neighborhood,Query,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,9,M1B,Scarborough,Malvern,"M1B, Scarborough, Malvern",43.8092,-79.2217,1,Fast Food Restaurant,Pizza Place,Pharmacy,Convenience Store,Grocery Store,Park,Sandwich Place,Supermarket,Bubble Tea Shop,Skating Rink
1,18,M1C,Scarborough,Rouge Hill,"M1C, Scarborough, Rouge Hill",43.7803,-79.1305,4,Train Station,Bus Line,Wine Bar,Furniture / Home Store,Diner,Discount Store,Dive Bar,Electronics Store,Factory,Fast Food Restaurant
2,26,M9C,Etobicoke,Eringate,"M9C, Etobicoke, Eringate",43.6623,-79.5765,2,Electronics Store,Park,Wine Bar,Furniture / Home Store,Diner,Discount Store,Dive Bar,Factory,Fast Food Restaurant,Food Court
3,36,M1G,Scarborough,Woburn,"M1G, Scarborough, Woburn",43.7598,-79.2253,1,Fast Food Restaurant,Discount Store,Coffee Shop,Bank,Paper / Office Supplies Store,Clothing Store,Big Box Store,Beer Store,Sandwich Place,Electronics Store
4,49,M5H,Downtown Toronto,Richmond,"M5H, Downtown Toronto, Richmond",43.6512,-79.3813,1,Hotel,Clothing Store,Café,Coffee Shop,Gym,Restaurant,American Restaurant,Asian Restaurant,Bakery,Gastropub


In [60]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=15,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

In [50]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,M9R,43.6995,-79.5563,0,Park,Wine Bar,Furniture / Home Store,Diner,Discount Store,Dive Bar,Electronics Store,Factory,Fast Food Restaurant,Food Court


In [51]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,43.8092,-79.2217,1,Fast Food Restaurant,Pizza Place,Pharmacy,Convenience Store,Grocery Store,Park,Sandwich Place,Supermarket,Bubble Tea Shop,Skating Rink
3,M1G,43.7598,-79.2253,1,Fast Food Restaurant,Discount Store,Coffee Shop,Bank,Paper / Office Supplies Store,Clothing Store,Big Box Store,Beer Store,Sandwich Place,Electronics Store
4,M5H,43.6512,-79.3813,1,Hotel,Clothing Store,Café,Coffee Shop,Gym,Restaurant,American Restaurant,Asian Restaurant,Bakery,Gastropub
5,M6K,43.6509,-79.44,1,Bar,Vietnamese Restaurant,Park,Dive Bar,French Restaurant,Coffee Shop,Gastropub,Pizza Place,Portuguese Restaurant,Bakery
6,M2M,43.786,-79.416,1,Coffee Shop,Korean Restaurant,Japanese Restaurant,Sandwich Place,Grocery Store,Fried Chicken Joint,Pharmacy,Park,Restaurant,Gym
7,M2N,43.7615,-79.4109,1,Coffee Shop,Korean Restaurant,Japanese Restaurant,Sandwich Place,Grocery Store,Fried Chicken Joint,Pharmacy,Park,Restaurant,Gym
8,M6N,43.6682,-79.4793,1,Coffee Shop,Pizza Place,Park,Pharmacy,Athletics & Sports,Gas Station,Fried Chicken Joint,Grocery Store,Liquor Store,Factory
9,M6P,43.6539,-79.4669,1,Convenience Store,Pool,Gym,Deli / Bodega,Pizza Place,Pet Store,Café,Mexican Restaurant,Sandwich Place,Sporting Goods Shop
11,M6S,43.6635,-79.4895,1,Coffee Shop,Pizza Place,Park,Pharmacy,Athletics & Sports,Gas Station,Fried Chicken Joint,Grocery Store,Liquor Store,Factory


In [52]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M9C,43.6623,-79.5765,2,Electronics Store,Park,Wine Bar,Furniture / Home Store,Diner,Discount Store,Dive Bar,Factory,Fast Food Restaurant,Food Court


In [53]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,M1W,43.8162,-79.3145,3,Playground,Wine Bar,Furniture / Home Store,Diner,Discount Store,Dive Bar,Electronics Store,Factory,Fast Food Restaurant,Food Court


In [54]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M1C,43.7803,-79.1305,4,Train Station,Bus Line,Wine Bar,Furniture / Home Store,Diner,Discount Store,Dive Bar,Electronics Store,Factory,Fast Food Restaurant
